**Data Preprocessing for SVM Based Depression Prediction Model**

***1.Import Required Libraries***

In [57]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split


***2.Load the Dataset***

In [59]:
#Load the dataset
#file_path = os.path.join(os.getcwd(), "Data", "raw", "student_depression_dataset.csv")
#df = pd.read_csv(file_path)

df=pd.read_csv("./Data/raw/student_depression_dataset.csv")

In [60]:
#Preview the data
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,'5-6 hours',Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,'5-6 hours',Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,'Less than 5 hours',Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,'7-8 hours',Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,'5-6 hours',Moderate,M.Tech,Yes,1.0,1.0,No,0


***3.Inspect the Dataset***

In [61]:
# Print basic info about the dataset
print(f"   Information of the raw dataset \n -------------------------------------------\n")
df.info()

   Information of the raw dataset 
 -------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27901 entries, 0 to 27900
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     27901 non-null  int64  
 1   Gender                                 27901 non-null  object 
 2   Age                                    27901 non-null  float64
 3   City                                   27901 non-null  object 
 4   Profession                             27901 non-null  object 
 5   Academic Pressure                      27901 non-null  float64
 6   Work Pressure                          27901 non-null  float64
 7   CGPA                                   27901 non-null  float64
 8   Study Satisfaction                     27901 non-null  float64
 9   Job Satisfaction                       27901 non-null  fl

***4.Check whether there are missing values in the dataset***

In [62]:
#Check missing values
print("Missing values in each Column:\n")
print(df.isnull().sum())

Missing values in each Column:

id                                       0
Gender                                   0
Age                                      0
City                                     0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         0
Family History of Mental Illness         0
Depression                               0
dtype: int64


***5.Check whether there are duplicates in the dataset***

In [66]:
#Check for duplicate rows
print("Number of Duplicate Rows:/n")
print(df.duplicated().sum())

Number of Duplicate Rows:/n
0


***6.Drop Irrelevant Columns***

In [64]:
#Drop the 'id' column as it is not useful for prediction
df.drop(columns=['id'],inplace=True)

***7.Identify Feature Types***

In [65]:
#Target Variable/column
target='Depression'

#Feature columns(excluding target)
features=df.drop(columns=[target])

#Categorical Features
categorical_cols=features.select_dtypes(include='object').columns.tolist()

#Numerical Features
numerical_cols=features.select_dtypes(exclude='object').columns.tolist()

print("Categorical:",categorical_cols)
print("Numerical:",numerical_cols)

Categorical: ['Gender', 'City', 'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?', 'Financial Stress', 'Family History of Mental Illness']
Numerical: ['Age', 'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction', 'Work/Study Hours']


***8.Encode Categorical Variables***

SVM only works with numerical data. So need to encode categorical data.

*8.1.Label Encode Binary Categorical Columns*

In [50]:
le = LabelEncoder()


#List of binary categorical columns
binary_cols=['Gender','Academic Pressure','Work Pressure',
               'Study Satisfaction','Job Satisfaction',
               'Dietary Habits','Have you ever had suicidal thoughts ?',
               'Financial Stress','Family History of Mental Illness']

#Apply Label Encoding
for col in binary_cols:
    df[col] =le.fit_transform(df[col])

*8.2.One-Hot encode nominal categorical Columns*

In [51]:
#Apply One-Hot Encoding to niminal(multi-categorical) columns
df=pd.get_dummies(df,columns=['City','Profession','Degree'],drop_first=True)

*8.3.Encode the Target Variable*

In [52]:
#Convert the target variable(Depression) to numeric
df['Depression']=le.fit_transform(df['Depression'])

***9.Convert Original Range columns to Numeric***

Columns like 'Sleep Duration' and 'Work/Study Hours' contain rages which are strings. So, we need to map them to numerical values.

In [53]:
# Convert 'Sleep Duration' and 'Work/Study Hours' to numeric
sleep_map={
    'less than 4 hours':3,
    '4-5 hours':4.5,
    '5-6 hours':5.5,
    '6-7 hours':6.5,
    '7-8 hours':7.5,
    'more than 8 hours':9
}

work_study_map={
    'less than 4 hours':3,
    '4-6 hours':5,
    '6-8 hours':7,
    'more than 8 hours':9
}

#Apply the mappings
df['Sleep Duration']=df['Sleep Duration'].map(sleep_map)
df['Work/Study Hours']=df['Work/Study Hours'].map(work_study_map)


***10.Feature Scaling***

In [54]:
# Separate features and target
X=df.drop('Depression',axis=1)
y=df['Depression']

# Scale the features
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)


/usr/local/lib/python3.10/site-packages/sklearn/utils/extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/site-packages/sklearn/utils/extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/site-packages/sklearn/utils/extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [ ]:
# Convert to DataFrame to see the scaled values with column names
X_scaled_df=pd.DataFrame(X_scaled, columns=X.columns)

# Display first 5 rows
X_scaled_df.head()

,Gender,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Have you ever had suicidal thoughts ?,...,Degree_MA,Degree_MBA,Degree_MBBS,Degree_MCA,Degree_MD,Degree_ME,Degree_MHM,Degree_MSc,Degree_Others,Degree_PhD
0,0.891416,1.463165,1.345543,-0.009978,0.893393,-0.693425,-0.01534,NaN,-1.182363,0.761745,...,-0.141015,-0.143376,-0.159949,-0.197161,-0.144673,-0.0817,-0.083023,-0.211071,-0.03544,-0.138079
1,-1.121811,-0.371474,-0.826104,-0.009978,-1.194076,1.510636,-0.01534,NaN,-0.375705,-1.312775,...,-0.141015,-0.143376,-0.159949,-0.197161,-0.144673,-0.0817,-0.083023,-0.211071,-0.03544,-0.138079
2,0.891416,1.055467,-0.102222,-0.009978,-0.425724,1.510636,-0.01534,NaN,-1.182363,-1.312775,...,-0.141015,-0.143376,-0.159949,-0.197161,-0.144673,-0.0817,-0.083023,-0.211071,-0.03544,-0.138079
3,-1.121811,0.443921,-0.102222,-0.009978,-1.404862,-0.693425,-0.01534,NaN,-0.375705,0.761745,...,-0.141015,-0.143376,-0.159949,-0.197161,-0.144673,-0.0817,-0.083023,-0.211071,-0.03544,-0.138079
4,-1.121811,-0.167625,0.621660,-0.009978,0.322229,0.041262,-0.01534,NaN,-0.375705,0.761745,...,-0.141015,-0.143376,-0.159949,-0.197161,-0.144673,-0.0817,-0.083023,-0.211071,-0.03544,-0.138079


***11.Split Data into Train and Test Sets**

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(
    X_scaled,y,test_size=0.2,random_state=42,stratify=y
)

# Show shapes
print("Training set shape:",X_train.shape)
print("Testing set shape:",X_test.shape)

Training set shape: (22320, 104)
Testing set shape: (5581, 104)
